In [1]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    print('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')


In [80]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.getOrCreate()

In [81]:
df = spark.read.parquet('work/washing.parquet')
df.createOrReplaceTempView('washing') 
df.show() 

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|0d86485d0f88d1f9d...|1-57940679fb8a713...|    4|      11|acceptable|     NULL|      77| NULL|        100|1547808723923|   NULL|
|0d86485d0f88d1f9d...|1-15ff3a0b304d789...|    2|    NULL|      NULL|     NULL|    NULL| 1046|       NULL|1547808729917|   NULL|
|0d86485d0f88d1f9d...|1-97c2742b68c7b07...|    4|    NULL|      NULL|       71|    NULL| NULL|       NULL|1547808731918|    236|
|0d86485d0f88d1f9d...|1-eefb903dbe45746...|   19|      11|acceptable|     NULL|      75| NULL|         86|1547808738999|   NULL|
|0d86485d0f88d1f9d...|1-5f68b4c72813c25...|    7|    NULL|      NULL|       75|    NULL| NULL|   

# Assigment 2

In [82]:
def count():
    return spark.sql('SELECT COUNT(*) as cnt FROM washing').first().cnt

In [83]:
def getNumberOfFields():
    return len(df.columns)

In [84]:
def getFieldNames():
    return df.columns

In [85]:
cnt = count()
nof = getNumberOfFields()
fn = getFieldNames()
print(
    """
    Количество строк: {}
    Количество столбцов: {}
    Имена столбцов: {}
    """.format(cnt, nof, fn)
    )


    Количество строк: 2058
    Количество столбцов: 11
    Имена столбцов: ['_id', '_rev', 'count', 'flowrate', 'fluidlevel', 'frequency', 'hardness', 'speed', 'temperature', 'ts', 'voltage']
    


# Assigment 3

In [86]:
def minTemperature():
    return spark.sql("SELECT MIN(temperature) as mintemp FROM washing").first().mintemp

In [87]:
def meanTemperature():
    return spark.sql("SELECT MEAN(temperature) as meantemp from washing").first().meantemp

In [88]:
def maxTemperature():
    return spark.sql("SELECT MAX(temperature) as maxtemp FROM washing").first().maxtemp

In [89]:
def sdTemperature():
    return spark.sql("SELECT STDDEV_POP(temperature) as sdtemp FROM washing").first().sdtemp

In [90]:
def skewTemperature():    
    return spark.sql("""
SELECT 
    (
        1/COUNT(*)
    ) *
    SUM (
        POWER(temperature-%s,3)/POWER(%s,3)
    )

as sktemperature from washing
                    """ %(meanTemperature(),sdTemperature())).first().sktemperature

In [91]:
def kurtosisTemperature():    
        return spark.sql("""
SELECT 
    (
        1/COUNT(*)
    ) *
    SUM (
        POWER(temperature-%s,4)/POWER(%s,4)
    )
as ktemperature from washing
                    """ %(meanTemperature(),sdTemperature())).first().ktemperature


In [92]:
def correlationTemperatureHardness():
    return spark.sql("SELECT CORR(temperature,hardness) as temperaturehardness from washing").first().temperaturehardness

In [93]:
min_temperature = minTemperature()
mean_temperature = meanTemperature()
max_temperature = maxTemperature()
sd_temperature = sdTemperature()
skew_temperature = skewTemperature()
kurtosis_temperature = kurtosisTemperature()
correlation_temperature = correlationTemperatureHardness()

print(
    """
    Минимальная температура: {}
    Средняя температура: {}
    Максимальная температура: {}
    Стандартное отклонение температуры: {}
    Асимметрия температуры: {}
    Эксцесс температуры: {}
    Корреляция температуры и жесткости: {}
    """.format(min_temperature,
         mean_temperature,
         max_temperature,
         sd_temperature,
         skew_temperature,
         kurtosis_temperature,
         correlation_temperature)
)



    Минимальная температура: 80
    Средняя температура: 90.03800298062593
    Максимальная температура: 100
    Стандартное отклонение температуры: 6.098487624200337
    Асимметрия температуры: 0.006788255973582835
    Эксцесс температуры: 1.158158434967638
    Корреляция температуры и жесткости: 0.017754069047296324
    
